<a href="https://colab.research.google.com/github/Russele7/python_for_microscopists/blob/master/python_and_sql_dml_use_cases.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import seaborn as sns

import sqlite3

In [0]:
titanic = sns.load_dataset("titanic")

In [0]:
sns.get_dataset_names()

/usr/local/lib/python3.6/dist-packages/seaborn/utils.py:384: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 384 of the file /usr/local/lib/python3.6/dist-packages/seaborn/utils.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  gh_list = BeautifulSoup(http)


['anscombe',
 'attention',
 'brain_networks',
 'car_crashes',
 'diamonds',
 'dots',
 'exercise',
 'flights',
 'fmri',
 'gammas',
 'iris',
 'mpg',
 'planets',
 'tips',
 'titanic']

In [0]:
titanic.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [0]:
titanic["who"].value_counts()

man      537
woman    271
child     83
Name: who, dtype: int64

In [0]:
titanic["sex"].value_counts()

male      577
female    314
Name: sex, dtype: int64

In [0]:
conn = sqlite3.connect('example.db')

In [0]:
titanic.to_sql("titanic", conn)

In [0]:
titanic.to_sql()

# Вариант №1
1. Пишем SQL выражение
2. Загружаем данные из таблицы

In [0]:
sql_statement = """SELECT
  sex
, AVG(age) AS age_mean
, SUM(survived) AS survived
FROM titanic
WHERE age > 10
GROUP BY sex
"""

titanic_stat = pd.read_sql(sql_statement, conn)

In [0]:
titanic_stat.head()

,sex,age_mean,survived
0,female,31.058696,178
1,male,32.829762,74


# Вариант №2
Для повторяющихся кейсов, лучше всего будет автоматизировать такие операции и создать функцию.


In [0]:
def get_titanic_stat(by="sex", age=10):
  """Get statistics from titanic

  Args:
    sex
    age
  
  Returns:
    
  """
  sql_template = """SELECT
  {0}
, AVG(age) AS age_mean
, SUM(survived) AS survived
FROM titanic
WHERE age > {1}
GROUP BY {0}
"""
  return pd.read_sql(
      sql_template.format(by, age),
      conn
  )

In [0]:
import datetime

In [0]:
test_string = "time: {0}"
test_string.format(datetime.datetime.now())

'time: 2020-05-23 08:06:43.318513'

In [0]:
test_string = "time: {time}"
test_string.format(time=datetime.datetime.now())

'time: 2020-05-23 08:07:53.184747'

In [0]:
time = datetime.datetime.now()

test_string = f"time: {time}"
test_string

'time: 2020-05-23 08:08:46.270600'

In [0]:
titanic_stat = get_titanic_stat("pclass", 10)

In [0]:
titanic_stat.head()

,pclass,age_mean,survived
0,1,38.822404,120
1,2,32.753205,66
2,3,28.032154,66


# Вариант №3
Обратите внимание, что мы можем передать аргументы (в нашем случае это _age_) не только с помощью строкового форматирования, а с помощью аргумента params. Это называется передать параметры (bind params).

In [0]:
def get_titanic_stat(by="sex", age=10):
  sql_template = """SELECT
  {0}
, AVG(age) AS age_mean
, SUM(survived) AS survived
FROM titanic
WHERE age > :age
GROUP BY {0}
"""
  return pd.read_sql(
      sql_template.format(by),
      conn,
      params={"age": age}
  )

In [0]:
titanic_stat = get_titanic_stat("pclass", 10)

In [0]:
titanic_stat.head()

,pclass,age_mean,survived
0,1,38.822404,120
1,2,32.753205,66
2,3,28.032154,66


# Вариант №4
Описать таблицы как Объекты и обращаться с ними как с объектом Python. Для этого я использую ORM SQLAlchemy

__Здесь заранее извиняюсь__ - код, который отлично работает в Oracle с примером на SQLite у меня отказывается работать, поэтому пример немного упрощенный. PS - SQLite используется исключительно в целях иллюстрации - редко где на практике видел использование этой СУБД кроме как в примерах

In [0]:
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import MetaData
from sqlalchemy.orm import sessionmaker
from sqlalchemy import Table, Column, Integer, Numeric, String, ForeignKey, DateTime
from sqlalchemy import func

metadata = MetaData()

In [0]:
Base = declarative_base()

# Опишем нашу таблицу
class Titanic(Base):
    __tablename__ = 'titanic'

    id = Column(Integer, primary_key=True)
    sex = Column(String)
    pclass = Column(Integer)
    age = Column(Integer)
    survived = Column(Integer)

In [0]:
def get_titanic_stat():
  # Создадим сессию
  Session = sessionmaker(bind=conn)
  session = Session()
  query = session.query(
      Titanic.pclass,
      func.sum(Titanic.survived)
  ).group_by(Titanic.pclass)

  print(query.statement)
  
  return pd.read_sql(str(query.statement), conn)

In [0]:
titanic_stat = get_titanic_stat()

SELECT titanic.pclass, sum(titanic.survived) AS sum_1 
FROM titanic GROUP BY titanic.pclass


In [0]:
titanic_stat.head()

,pclass,sum_1
0,1,136
1,2,87
2,3,119


# Вариант №5
Использование шаблонизатора на основе Jinja2 - jinjasql

In [0]:
!pip install jinjasql

In [0]:
from jinjasql import JinjaSql

In [0]:
def get_titanic_stat(age=10):
  sql_template = """SELECT
  sex
, AVG(age) AS age_mean
, SUM(survived) AS survived
FROM titanic
WHERE age > {{ age }}
GROUP BY sex
"""
  j = JinjaSql(param_style="named")
  query, params = j.prepare_query(sql_template, {"age": age})
  return pd.read_sql(query, conn, params=params)

In [0]:
titanic_stat = get_titanic_stat(age=10)

In [0]:
titanic_stat.head()

,sex,age_mean,survived
0,female,31.058696,178
1,male,32.829762,74


Думаю, что для того, чтобы закрыть большую часть ваших потребностей в автоматизации работы в DML (SELECT) в Python - этого хватит. Экспериментируйте.